In [1]:
import os
from dotenv import load_dotenv

In [2]:
# Load the API key from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if OPENAI_API_KEY:
    print("API key loaded successfully.")
else:
    raise ValueError("Error loading API key. Check that OPENAI_API_KEY is set inside .env")

API key loaded successfully.


## Few-Shot Prompting
https://python.langchain.com/docs/how_to/few_shot_examples/

In [3]:
from langchain.prompts import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

### Example Set

In [4]:
example_prompt = PromptTemplate.from_template("Question: {input}\n{output}")

In [5]:
examples = [
    {
        "input": "Block my son's computers from malicious websites.",
        "output": """
<i2nsf-cfi-policy
    xmlns="urn:ietf:params:xml:ns:yang:ietf-i2nsf-cfi-policy">
    <name>block_web_security_policy</name>
    <rules>
        <name>block_web</name>
        <condition>
            <firewall-condition>
                <source>Son's_PC</source>
            </firewall-condition>
            <url-condition>
                <url-name>malicious_websites</url-name>
            </url-condition>
        </condition>
        <actions>
            <primary-action>
                <action>drop</action>
            </primary-action>
        </actions>
    </rules>
</i2nsf-cfi-policy>
        """
    },
    {
        "input": "Restrict access to adult content from all devices.",
        "output": """
<i2nsf-cfi-policy
    xmlns="urn:ietf:params:xml:ns:yang:ietf-i2nsf-cfi-policy">
    <name>restrict_adult_content_policy</name>
    <rules>
        <name>restrict_adult</name>
        <condition>
            <url-condition>
                <url-name>adult_content</url-name>
            </url-condition>
        </condition>
        <actions>
            <primary-action>
                <action>drop</action>
            </primary-action>
        </actions>
    </rules>
</i2nsf-cfi-policy>
        """
    }
]


In [6]:
print(example_prompt.invoke(examples[0]).to_string())

Question: Block my son's computers from malicious websites.

<i2nsf-cfi-policy
    xmlns="urn:ietf:params:xml:ns:yang:ietf-i2nsf-cfi-policy">
    <name>block_web_security_policy</name>
    <rules>
        <name>block_web</name>
        <condition>
            <firewall-condition>
                <source>Son's_PC</source>
            </firewall-condition>
            <url-condition>
                <url-name>malicious_websites</url-name>
            </url-condition>
        </condition>
        <actions>
            <primary-action>
                <action>drop</action>
            </primary-action>
        </actions>
    </rules>
</i2nsf-cfi-policy>
        


### Example Selector

In [9]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)

# Select the most similar example to the input.
input = "Block all games during the nights at weekdays"
selected_examples = example_selector.select_examples({"input": input})
print(f"Examples most similar to the input: {input}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Block all games during the nights at weekdays


input: Block my son's computers from malicious websites.
output: 
<i2nsf-cfi-policy
    xmlns="urn:ietf:params:xml:ns:yang:ietf-i2nsf-cfi-policy">
    <name>block_web_security_policy</name>
    <rules>
        <name>block_web</name>
        <condition>
            <firewall-condition>
                <source>Son's_PC</source>
            </firewall-condition>
            <url-condition>
                <url-name>malicious_websites</url-name>
            </url-condition>
        </condition>
        <actions>
            <primary-action>
                <action>drop</action>
            </primary-action>
        </actions>
    </rules>
</i2nsf-cfi-policy>
        


### FewShotPromptTemplate

In [18]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Generate and return only the security policies in XML format based on the user's input.",
    suffix="Input: {input}",
    input_variables=["input"],
)

print(
    few_shot_prompt.invoke({"input": "Block all games during the nights at weekdays"}).to_string()
)

Generate and return only the security policies in XML format based on the user's input.

Question: Block my son's computers from malicious websites.

<i2nsf-cfi-policy
    xmlns="urn:ietf:params:xml:ns:yang:ietf-i2nsf-cfi-policy">
    <name>block_web_security_policy</name>
    <rules>
        <name>block_web</name>
        <condition>
            <firewall-condition>
                <source>Son's_PC</source>
            </firewall-condition>
            <url-condition>
                <url-name>malicious_websites</url-name>
            </url-condition>
        </condition>
        <actions>
            <primary-action>
                <action>drop</action>
            </primary-action>
        </actions>
    </rules>
</i2nsf-cfi-policy>
        

Input: Block all games during the nights at weekdays


### LLM

In [19]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [21]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
print(few_shot_prompt.invoke(input="Block all games during the nights at weekdays").to_messages()[-1].content)

Generate and return only the security policies in XML format based on the user's input.

Question: Block my son's computers from malicious websites.

<i2nsf-cfi-policy
    xmlns="urn:ietf:params:xml:ns:yang:ietf-i2nsf-cfi-policy">
    <name>block_web_security_policy</name>
    <rules>
        <name>block_web</name>
        <condition>
            <firewall-condition>
                <source>Son's_PC</source>
            </firewall-condition>
            <url-condition>
                <url-name>malicious_websites</url-name>
            </url-condition>
        </condition>
        <actions>
            <primary-action>
                <action>drop</action>
            </primary-action>
        </actions>
    </rules>
</i2nsf-cfi-policy>
        

Input: Block all games during the nights at weekdays


In [22]:
chain = few_shot_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

print(chain.invoke({"input": "Block all games during the nights at weekdays"}).content)

```xml
<i2nsf-cfi-policy
    xmlns="urn:ietf:params:xml:ns:yang:ietf-i2nsf-cfi-policy">
    <name>block_games_night_policy</name>
    <rules>
        <name>block_games_night</name>
        <condition>
            <firewall-condition>
                <source>All_Devices</source>
            </firewall-condition>
            <time-condition>
                <day>Monday</day>
                <day>Tuesday</day>
                <day>Wednesday</day>
                <day>Thursday</day>
                <day>Friday</day>
                <time>22:00-23:59</time>
            </time-condition>
            <application-condition>
                <application-name>games</application-name>
            </application-condition>
        </condition>
        <actions>
            <primary-action>
                <action>drop</action>
            </primary-action>
        </actions>
    </rules>
</i2nsf-cfi-policy>
```
